In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys
import os
import time
import numpy as np

In [50]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction
snopt = SnoptSolver()
def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

# Build simulation Environment

In [51]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [52]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
        if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)

    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(models[0].model_instance):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


# Construct Rational Kinematics

In [53]:
from pydrake.all import RationalForwardKinematics
forward_kin = RationalForwardKinematics(plant)
q_star = np.zeros(forward_kin.t().shape[0])
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret


# Define Iris In T-Space

In [70]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()

vis.delete()

def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)

dReal_polytope_tol = .1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    print(type(X_WA))
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
    return result.is_success(), result.GetSolution(q)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b, guess=None):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")

    if guess is not None:
        prog.SetInitialGuess(q, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)

    # TODO: Remove these.  They're for debugging only.
    #set_meshcat_object(vis['setA'],setA)
    #set_meshcat_object(vis['setB'],setB)
    print(type(p_AA[0]))
    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        # TODO: Update this to use MBP<double> and Jacobians if I'm going to keep it.
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
#     vis = Visualizer(zmq_url=zmq_url)
#     set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
#     assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
#     set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(lb, ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = False
    use_dReal = False
    ad_bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    if use_autodiff:
        bodies = ad_bodies
    else:
        bodies = sym_bodies
    print
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                success, growth, qstar = GrowthVolume(E,
                    bodies[geomA], bodies[geomB],
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if use_dReal:
                tries = 0
                while True:
                    reachable, counter_example = CheckVolume(
                        E, sym_bodies[geomA], sym_bodies[geomB], sets[geomA], sets[geomB],
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, np.inf)
                    if not reachable:
                        print("unreachable")
                        break
                    else:
                        z = E.A() @ (counter_example - E.center())
                        dreal_growth = z.dot(z)
                        print(f"counter_example = {counter_example}, growth = {dreal_growth}")
                        success, growth, qstar = GrowthVolume(E,
                            bodies[geomA], bodies[geomB],
                            sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, counter_example)
                        if success:
                            print(f"snopt_example={qstar}, growth = {growth}")
                            # Add a face to the polytope
                            A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                            num_faces += 1
                        if np.all(A[:num_faces,:] @ counter_example <= b[:num_faces] - dReal_polytope_tol):
                            # Then also add the counter-example
                            A[num_faces,:], b[num_faces] = E.TangentPlane(counter_example)
                            num_faces += 1
                    tries += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
#         set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.MaximumVolumeInscribedEllipsoid()
#         set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


In [84]:
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
  
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)

    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
        
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [85]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = .1
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
    X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

#     link_poses_by_body_index = [RationalForwardKinematicsPoseToDrakePose(pose) for pose in link_poses_by_body_index_rat_pose]

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [86]:
seed_points = np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

In [87]:
def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)


In [89]:
#Currently SegFaulting

query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
regions = []
for i in range(seed_points.shape[0]):
    
    start_time = time.time()
# #     hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
#     hpoly = iris_cspace(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
#     regions.append(hpoly)
print("SUCCESS")

geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.26672972 -0.41852535  1.63205081], growth = 274.0684135179604
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.31188785 -0.73080518  0.69528238], growth = 338.66769414844066
geomA=wsg::right_collision, geomB=shelves::shelf_upper
geomA=iiwa::link7, geomB=shelves::shelf_lower
snopt_example=[ 0.52015706 -0.77647848  1.63205081], growth = 192.45911044623415
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=wsg::left_collision, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::body_collision, geomB=shelves::shelf_upper
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.

In [76]:
point = seed_points[0,:]
ellipsoid_epsilon = 1e-1
dim = plant.num_positions()
lb = plant.GetPositionLowerLimits()
rational_lb = forward_kin.ComputeTValue(lb, q_star)
ub = plant.GetPositionUpperLimits()
rational_ub = forward_kin.ComputeTValue(ub, q_star)
volume_of_unit_sphere = 4.0*np.pi/3.0
E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                     plant.world_body().index())
inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()

P = HPolyhedron.MakeBox(rational_lb, rational_ub)
A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

geomA, geomB = next(iter(pairs))
setA = sets[geomA]
setB = sets[geomB]
num_faces = 2*len(lb)
A = A[:num_faces,:]
b = b[:num_faces] - 1e-2


X_WA_Pose = link_poses_by_body_index_rat_pose[int(body_indexes_by_geom_id[geomA])]
X_WB_Pose = link_poses_by_body_index_rat_pose[int(body_indexes_by_geom_id[geomB])]

guess = None

In [77]:

X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]

def transform_replace_var(X, new_vars):
    R = X.rotation()
    Rmat = R.matrix()
    p = X.translation()
    rules = {t[i]: new_vars[i] for i in range(len(new_vars))}
    for i in range(Rmat.shape[0]):
        p[i] = p[i].Evaluate(rules)
        for j in range(Rmat.shape[1]):
            Rmat[i,j] = Rmat[i,j].Evaluate(rules)
    R.set(Rmat)
    return Rmat,p

In [80]:
prog = MathematicalProgram()
# s = prog.NewContinuousVariables(plant.num_positions(), "s")
t = forward_kin.t()
prog.AddDecisionVariables(t)
# X_WA_s = transform_replace_var(X_WA, s)
# X_WB_s = transform_replace_var(X_WB, s)

# R_WA, p_WA = transform_replace_var(X_WA, t)
# R_WB, p_WB = transform_replace_var(X_WB, t)


if guess is not None:
    prog.SetInitialGuess(t, guess)

prog.AddLinearConstraint(A, b-np.inf, b, t)
p_AA = prog.NewContinuousVariables(3, "p_AA")
p_BB = prog.NewContinuousVariables(3, "p_BB")
setA.AddPointInSetConstraints(prog, p_AA)
setB.AddPointInSetConstraints(prog, p_BB)
prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

p_WA = X_WA.multiply(p_AA+0)
        
p_WB = X_WB.multiply(p_BB+0)

prog.AddConstraint(eq(p_WA, p_WB))
        
# f = lambda x: x.ToExpression()
# vfunc = np.vectorize(f)
# R_WA_expr = vfunc(X_WA_Pose.rotation())
# R_WB_expr = vfunc(X_WB_Pose.rotation())
# p_WA_expr = vfunc(X_WA_Pose.translation())
# p_WB_expr = vfunc(X_WB_Pose.translation())


def kinematics_constraint(vars):
    p_AA, p_BB, s = np.split(vars,[3,6])

    R_WA, p_WA = transform_replace_var(X_WA, s)
    R_WB, p_WB = transform_replace_var(X_WB, s)
    p_WA = R_WA@p_AA + p_WA# X_WA_s.multiply(p_AA+0)# R_WA_expr@p_AA + p_WA_expr#
    p_WB = R_WB@p_BB + p_WB# X_WB_s.multiply(p_BB+0) #R_WB_expr@p_BB + p_WB_expr#
    
#     p_WA = X_WA.multiply(p_AA+0)# R_WA_expr@p_AA + p_WA_expr#
#     p_WB = X_WB.multiply(p_BB+0) 
    
#     p_WB_s = np.zeros_like(p_WB_t)
#     for i in range(s.shape[0]):
#         p_WA_s[i] = xreplace(p_WA_t[i], list(zip(forward_kin.t(), s)))
#         p_WB_s[i] = xreplace(p_WB_t[i], list(zip(forward_kin.t(), s)))
    return p_WA - p_WB

# tmp = (kinematics_constraint(np.concatenate((p_AA,p_BB,s))))
# prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
#                        vars=np.concatenate((p_AA,p_BB,t)))
# result = snopt.Solve(prog)


In [42]:
# prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
#                        vars=np.concatenate((p_AA,p_BB,t)))

<pydrake.solvers.mathematicalprogram.Binding[Constraint] at 0x7fc99b090030>

In [83]:
print(prog)
from pydrake.all import SolverOptions, CommonSolverOption
solver_options = SolverOptions()
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 1)


Decision variables:   t[0]    t[1]    t[2] p_AA(0) p_AA(1) p_AA(2) p_BB(0) p_BB(1) p_BB(2)

QuadraticCost (796.48810000000003 + 403.19999999999999 * t[1] - 395 * t[2] + 100 * pow(t[0], 2) + 100 * pow(t[1], 2) + 100 * pow(t[2], 2))
ExpressionConstraint
0 <= (-0.80000000000000004 - p_BB(0) + (p_AA(0) * ((-4.8963439307869184e-12 + 1.5314274795695805e-15 * t[0] + 1.1993183748638605e-27 * t[1] - 1.1993183748638605e-27 * t[2] - 1.958586558186733e-11 * (t[0] * t[1]) - 4.7972734994554386e-27 * (t[0] * t[1] * t[2]) + 1.958684529930665e-11 * (t[0] * t[1] * pow(t[2], 2)) + 1.531427479571979e-15 * (t[0] * pow(t[1], 2)) - 1.958635544058699e-11 * (t[0] * pow(t[1], 2) * t[2]) + 1.5314274795695805e-15 * (t[0] * pow(t[1], 2) * pow(t[2], 2)) + 1.958635544058699e-11 * (t[0] * t[2]) + 1.531427479571979e-15 * (t[0] * pow(t[2], 2)) - 1.1993183748638587e-27 * (pow(t[0], 2) * t[1]) + 1.958635544058699e-11 * (pow(t[0], 2) * t[1] * t[2]) + 1.1993183748638587e-27 * (pow(t[0], 2) * t[1] * pow(t[2], 2)) - 4.896343

In [82]:

result = snopt.Solve(prog, guess, solver_options)
print(result.is_success())

True


In [ ]:
f = lambda x: x.ToExpression()
vfunc = np.vectorize(f)
R_WA_expr = vfunc(X_WA_Pose.rotation())#np.array(map(f, X_WA_Pose.rotation()))
print(X_WA_Pose.rotation())
print(R_WA_expr@p_AA)

In [ ]:
# #unpack a symbolic geometry
# query_object = plant.get_geometry_query_input_port().Eval(plant_context)
# inspector = query_object.inspector()
# pairs = inspector.GetCollisionCandidates()
# sym_plant = plant.ToSymbolic()
# sym_context = sym_plant.CreateDefaultContext()
# geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
# sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
# geomA = iter(pairs).__next__()[0]
# frameA = sym_bodies[geomA]
# # prog = MathematicalProgram()

# # q = sym.MakeVectorContinuousVariable(3,"q")
# # t = prog.NewContinuousVariables(q.shape[0], "t")
# # trig_to_rat_rules = generate_rationalize_trig_expr_rules(q,t)

# # sym_plant.SetPositions(sym_context, q)
# # X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, frameA)
# # p_AA = prog.NewContinuousVariables(3, "p_AA")
# # #don't delete the plus. causes segfault of kernel
# # p_WA = X_WA.multiply(p_AA+0)

# # for i in range(p_WA.shape[0]):
# #     p_WA[i] = p_WA[i] #.Expand()
# # print(p_WA[0])



In [ ]:
# for geom in geom_ids:
#     body = plant.GetBodyFromFrameId(inspector.GetFrameId(geom))
#     body_index = body.index()
#     poses = forward_kin.CalcLinkPoses(q_star, body_index)
#     print(poses[4].R_AB()[0,0].numerator())
#     print()

In [ ]:
# print(p_WA[0])
tmp = p_WA[0].Expand()
# print(X_WA.rotation()[0,0].GetVariables())
print(tmp)

In [ ]:
import numbers

        
def make_rational_function_from_expression(expr):
    # TODO handle nested fractions
    LegalPolyExpressionKind = [
        sym.ExpressionKind.Var,
        sym.ExpressionKind.Add,
        sym.ExpressionKind.Mul,
        sym.ExpressionKind.Div,
        sym.ExpressionKind.Pow,
        sym.ExpressionKind.Constant
    ]
    if isinstance(expr, (numbers.Number, np.number)):
        return sym.Polynomial(expr)

    expr_kind = expr.get_kind()
    
    if expr.is_polynomial():
        return sym.Polynomial(expr)
    elif expr_kind not in LegalPolyExpressionKind:
        raise NotRationalFunctionException(expr.to_string() + " is not rational")
    elif expr_kind == sym.ExpressionKind.Div:
        (ctor, (numerator, denominator)) = expr.Deconstruct()
        numerator = sym.Polynomial(numerator)
        denominator = sym.Polynomial(denominator)
        return RationalFunction(numerator, denominator)
    elif expr_kind in LegalPolyExpressionKind:
        (ctor, args) = expr.Deconstruct()
        if expr_kind == sym.ExpressionKind.Mul or expr_kind == sym.ExpressionKind.Pow:
            res = RationalFunction(1)
        else:
            res = RationalFunction(0)
        for e in args:
            res = ctor(res, make_rational_function_from_expression(e))
        return res
    
    else:
        raise NotRationalFunctionException(expr.to_string() + " is not rational but of type " + expr.get_kind())


def make_rational_function_from_expression_and_count_calls(expr, num_calls):
    print(num_calls)
    num_calls += 1
    # TODO handle nested fractions
    LegalPolyExpressionKind = [
        sym.ExpressionKind.Var,
        sym.ExpressionKind.Add,
        sym.ExpressionKind.Mul,
        sym.ExpressionKind.Div,
        sym.ExpressionKind.Pow,
        sym.ExpressionKind.Constant
    ]
    if isinstance(expr, (numbers.Number, np.number)):
        return sym.Polynomial(expr), num_calls
    
    expr_kind = expr.get_kind()
    
    if expr.is_polynomial():
        return sym.Polynomial(expr), num_calls
    elif expr_kind not in LegalPolyExpressionKind:
        raise NotRationalFunctionException(expr.to_string() + " is not rational")
    elif expr_kind == sym.ExpressionKind.Div:
        (ctor, (numerator, denominator)) = expr.Deconstruct()
        numerator = sym.Polynomial(numerator)
        denominator = sym.Polynomial(denominator)
        return RationalFunction(numerator, denominator), num_calls
    elif expr_kind in LegalPolyExpressionKind:
        (ctor, args) = expr.Deconstruct()
        if expr_kind == sym.ExpressionKind.Mul or expr_kind == sym.ExpressionKind.Pow:
            res = RationalFunction(1)
        else:
            res = RationalFunction(0)
        for e in args:
            
            tmp, num_calls = make_rational_function_from_expression_and_count_calls(e, num_calls)
            res = ctor(res, tmp)

        return res, num_calls
    
    else:
        raise NotRationalFunctionException(expr.to_string() + " is not rational but of type " + expr.get_kind())

        
def rationalize_trig_expr(expr, rules):
    return make_rational_function_from_expression(xreplace(expr, rules))

In [ ]:
expr_in_q = p_WA[0]
expr_in_t = xreplace(expr_in_q, trig_to_rat_rules)
ctor, args = expr_in_t.Deconstruct()
print(len(args))


In [ ]:

import time
# print(expr_in_q)
start_time = time.time()
rational_in_t, num_calls = make_rational_function_from_expression_and_count_calls(expr_in_t, 0)
print("Time: %6.2f" % (time.time() - start_time))
print(rational_in_t)


In [ ]:
num = rational_in_t.numerator()

In [ ]:
print(num.Degree())
print("done")

In [ ]:
coeffs = num.GetCoefficients()

In [ ]:
print("hi")